<img src=../../Img/sci.png>

### Финансовый университет

##  Инструментальная поддержка анализа финансово-экономических данных

# Тема 5. Работа с базами данных в Пайтон. Библиотека SQLAlchemy Core

Семинар<br>
10 апреля 2021 года<br>
24 апреля 2021 года <br>
Поток: ПМ18-1, ПМ18-2, ПМ18-3, ПМ18-4

Преподаватель: Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

Москва - 2021

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- <a href="http://insideairbnb.com/get-the-data.html">Inside Airbnb</a>

# Проект Антверпен

<img src="./Img/Antwerp_1.png">

## Задание
Используя исходные данные системы <a href="http://insideairbnb.com/get-the-data.html">*Airbnb*</a> в формате *csv* об объектах размещения туристов в Антверпене, создать структуру реляционной базы данных согласно схеме. Создать первичные и внешние ключи, а также другие необходимые ограничения. Наполнить базу данных сведениями об объектах размещения в Антверпене.

<img src=./Img/Schema.png>

# 1. Изучение источника данных
<img src="./Img/Antwerpen_4.png">

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("Data/listings_antwerp.csv")
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,50904,aplace/antwerp: cosy suite - fashion district,234077,Karin,NaN,Historisch Centrum,51.21825,4.39901,Hotel room,145,1,1,2015-05-06,0.01,4,226
1,67776,Beautiful Big House Center Antwerp,334804,Ann,NaN,Sint-Andries,51.21467,4.39225,Entire home/apt,50,7,9,2016-10-24,0.08,2,210
2,116134,Trendy Vacation Apartment Antwerp,586942,Paul,NaN,Eilandje,51.23051,4.40593,Entire home/apt,150,2,102,2020-11-16,0.93,1,351
3,224333,Large stylish room in 1930s house + garden,1167377,Geert,NaN,Deurne Zuid West,51.19772,4.45853,Private room,16,14,2,2020-07-04,0.02,2,310
4,224682,APARTMENT ROSCAM - OLD CENTRE ANTWERP,1263933,Kristien,NaN,Sint-Andries,51.21722,4.39790,Entire home/apt,70,2,344,2021-01-28,3.13,1,299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,48217347,Antwerp top location! Beautiful 1 BDR apartment,129071533,Funny,NaN,Eilandje,51.22729,4.40859,Entire home/apt,80,7,0,NaN,NaN,1,90
1712,48224026,New LUXE DESIGN apt in centre Het zuid with HO...,389076822,Jean-Paul From BnbSupport,NaN,Zuid,51.21135,4.39703,Entire home/apt,399,2,0,NaN,NaN,1,364
1713,48227279,"Recently renovated, light apartment at great spot",70627813,Enid,NaN,Nieuw - Kwartier Oost,51.18437,4.43935,Entire home/apt,45,30,0,NaN,NaN,1,245
1714,48261827,Design appartement op toplocatie,266540939,Katia,NaN,Theaterbuurt-Meir,51.21321,4.40922,Entire home/apt,145,7,0,NaN,NaN,1,87


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716 entries, 0 to 1715
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1716 non-null   int64  
 1   name                            1716 non-null   object 
 2   host_id                         1716 non-null   int64  
 3   host_name                       1716 non-null   object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   1716 non-null   object 
 6   latitude                        1716 non-null   float64
 7   longitude                       1716 non-null   float64
 8   room_type                       1716 non-null   object 
 9   price                           1716 non-null   int64  
 10  minimum_nights                  1716 non-null   int64  
 11  number_of_reviews               1716 non-null   int64  
 12  last_review                     14

Удалим столбец *neighbourhood_group*, так как он не содержит данных.

В поле *last_review* более 300 записей содержат пустые значения. Заменим все такие значения на дату 1 января 1990 года. 

Также имеются пустые значения в поле *reviews_per_month*. Такие заменим на нуль.

In [4]:
df.drop("neighbourhood_group", axis=1, inplace=True)
df["last_review"].fillna("1990-01-01", inplace=True)
df["reviews_per_month"].fillna(0, inplace=True)

# 2. Создание базы данных *Antwerp.db*

<img src="./Img/Antwerpen_3.png">

In [5]:
# ! pip install sqlalchemy

In [6]:
import sqlalchemy

from sqlalchemy import (create_engine, MetaData, Table, Column,
                        Integer, Numeric, String, Date, Boolean,
                        ForeignKey, PrimaryKeyConstraint, UniqueConstraint, 
                        CheckConstraint, ForeignKeyConstraint,
                        insert, select, func, desc)

from datetime import date, datetime as dt
from pprint import pprint

engine = create_engine('sqlite:///Antwerp.db')
metadata = MetaData()

In [7]:
listings = Table('listings', metadata,
            Column('listing_id',Integer(),primary_key=True),
            Column('listing_name',String(100),index=True),
            Column('host_id',Integer()),
            Column('neighbourhood_id',Integer()),
            Column('latitude',Numeric(10,2)),
            Column('longitude',Numeric(10,2)),
            Column('room_type_id',Integer()),
            Column('price',Integer()),
            Column('minimum_nights',Integer()),
            Column('number_of_reviews', Integer()),
            Column('last_review', Date()),
            Column('reviews_per_month', Numeric(10,2)),
            Column('availability_365', Integer()),
            ForeignKeyConstraint(['host_id'], ['hosts.host_id']),
            ForeignKeyConstraint(['neighbourhood_id'], ['neighbourhoods.neigh_id']),
            ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
            extend_existing=True
            )

In [8]:
users = Table('users',metadata,
            Column('user_id',Integer(),primary_key=True),
            Column('user_name',String(15),nullable=False,unique=True),
            Column('email_address',String(255),nullable=False),
            Column('phone',String(20),nullable=False),
            Column('password',String(25),nullable=False),
            Column('created_on',Date(),default=date.today()),
            Column('updated_on',Date(),default=date.today(),onupdate=date.today()),
            extend_existing=True
           )

In [9]:
orders = Table('orders', metadata,
            Column('order_id', Integer(),primary_key=True),
            Column('user_id', ForeignKey('users.user_id')),
            extend_existing=True
            )

In [10]:
line_items = Table('line_items', metadata,
                   Column('line_item_id', Integer(), primary_key=True),
                   Column('order_id', Integer()),
                   Column('listing_id', ForeignKey('listings.listing_id')),
                   Column('item_start_date', Date()),
                   Column('item_end_date', Date()),
                   Column('item_price', Integer()),
                   extend_existing=True
                  )

line_items.append_constraint(
    ForeignKeyConstraint(['order_id'], ['orders.order_id'])
)

In [11]:
neighbourhoods = Table('neighbourhoods',metadata,
                    Column('neigh_id',Integer(),primary_key=True),
                    Column('neigh_name',String(30)),
                    extend_existing=True
                   )

In [12]:
hosts = Table('hosts',metadata,
           Column('host_id',Integer(),primary_key=True),
           Column('host_name',String(30))
          )

In [13]:
room_types = Table('room_types',metadata,
                Column('room_type_id',Integer(),primary_key=True),
                Column('room_type_name',String(30))
               )

In [14]:
metadata.create_all(engine)

Проверим, какие таблицы созданы

In [15]:
metadata.tables.keys()

dict_keys(['listings', 'users', 'orders', 'line_items', 'neighbourhoods', 'hosts', 'room_types'])

In [16]:
pprint(metadata.tables)

{'hosts': Table('hosts', MetaData(bind=None), Column('host_id', Integer(), table=<hosts>, primary_key=True, nullable=False), Column('host_name', String(length=30), table=<hosts>), schema=None),
 'line_items': Table('line_items', MetaData(bind=None), Column('line_item_id', Integer(), table=<line_items>, primary_key=True, nullable=False), Column('order_id', Integer(), ForeignKey('orders.order_id'), table=<line_items>), Column('listing_id', Integer(), ForeignKey('listings.listing_id'), table=<line_items>), Column('item_start_date', Date(), table=<line_items>), Column('item_end_date', Date(), table=<line_items>), Column('item_price', Integer(), table=<line_items>), schema=None),
 'listings': Table('listings', MetaData(bind=None), Column('listing_id', Integer(), table=<listings>, primary_key=True, nullable=False), Column('listing_name', String(length=100), table=<listings>), Column('host_id', Integer(), ForeignKey('hosts.host_id'), table=<listings>), Column('neighbourhood_id', Integer(), Fo

# 3. Наполнение справочников

<img src="./Img/Antwerpen_2.png">

Соединение

In [17]:
connection = engine.connect()

## 3.1. Наполнение справочника районов

In [18]:
df.head(1)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,50904,aplace/antwerp: cosy suite - fashion district,234077,Karin,Historisch Centrum,51.21825,4.39901,Hotel room,145,1,1,2015-05-06,0.01,4,226


In [19]:
neigh_names = df["neighbourhood"].sort_values().unique()
neigh_list = []
for i, x in enumerate(neigh_names):
    neigh_dict = {'neigh_id':i+1, 'neigh_name':x}
    neigh_list.append(neigh_dict)
pprint(neigh_list[:2])
print("Всего районов:", len(neigh_list))

[{'neigh_id': 1, 'neigh_name': 'Amandus - Atheneum'},
 {'neigh_id': 2, 'neigh_name': 'Borgerhout Extra Muros'}]
Всего районов: 53


In [20]:
ins = neighbourhoods.insert()
connection.execute(ins, neigh_list)

Проверка - запрос к таблице.

In [109]:
sel=select([neighbourhoods])
connection.execute(sel).fetchone()

(1, 'Amandus - Atheneum')

## 3.2. Наполнение справочника типов комнат

In [22]:
room_type_names=df["room_type"].sort_values().unique()
rooms_list=[]
for i, name in enumerate(room_type_names):
    tmp_dict={"room_type_id":i+1, "room_type_name":name}
    rooms_list.append(tmp_dict)
print(rooms_list[:1])
print("Всего типов комнат:", len(rooms_list))

[{'room_type_id': 1, 'room_type_name': 'Entire home/apt'}]
Всего типов комнат: 4


In [23]:
ins=room_types.insert()
connection.execute(ins, rooms_list)

Проверка - запрос к таблице *room_types*

In [24]:
sel=select([room_types])
connection.execute(sel).fetchone()

(1, 'Entire home/apt')

## 3.3. Наполнение справочника владельцев

### Задание 3.3.1.
Наполните справочник владельцев недвижимости, сохранив их идентификаторы из набора данных *Антверпен*.

In [25]:
# Ваш код здесь


In [26]:
# Решение
hosts_df=df.groupby(['host_id','host_name']).host_id.count()
hosts_df

host_id    host_name                
234077     Karin                        4
334804     Ann                          2
462975     Els                          1
573329     Patricia                     1
586942     Paul                         1
                                       ..
384478659  Mike                         1
386268235  Monali                       2
386740674  Ingrid                       1
387215929  Sam                          1
389076822  Jean-Paul From BnbSupport    1
Name: host_id, Length: 1043, dtype: int64

In [27]:
hosts_list=[]
for item in hosts_df.index:
    tmp_dict={'host_id':item[0],'host_name':item[1]}
    hosts_list.append(tmp_dict)
hosts_list[:3]

[{'host_id': 234077, 'host_name': 'Karin'},
 {'host_id': 334804, 'host_name': 'Ann'},
 {'host_id': 462975, 'host_name': 'Els'}]

In [28]:
ins=hosts.insert()
connection.execute(ins, hosts_list)

In [29]:
# Проверка
sel=select([hosts])
sel=sel.where(hosts.c.host_id==586942)
print(str(sel))
print(sel.compile().params)
rp=connection.execute(sel).fetchall()
print(rp)

SELECT hosts.host_id, hosts.host_name 
FROM hosts 
WHERE hosts.host_id = :host_id_1
{'host_id_1': 586942}
[(586942, 'Paul')]


# 4. Наполнение таблиц
<img src="./Img/Antwerpen_5.png">

## 4.1.  Таблица *listings*

В процессе наполнения таблицы *listings* мы будем подставлять вместо названий районов, названий типов комнат и имен владельцев их идентификаторы. Для этого испоьзуем ранее созданные справочники районов, типов комнат и владельцев, откуда будем подставлять в таблицу соовтетствующие значения идентификаторов.

**Пример.** Пусть у нас имеется справочник типов комнат - таблица *room_types*. Посмотрим, какие она содержит записи.

In [30]:
sel = select([room_types])
result = connection.execute(sel).fetchall()
pprint(result)

[(1, 'Entire home/apt'),
 (2, 'Hotel room'),
 (3, 'Private room'),
 (4, 'Shared room')]


Теперь посмотрим, какой тип комнаты у первой записи таблицы набора данных "Антверпен".

In [31]:
df.loc[0, 'room_type']

'Hotel room'

Это `'Hotel room'`. Чтобы получить идентификатор этого типа комнаты выполним запрос:

In [32]:
sel = select([room_types.c.room_type_id])
sel = sel.where(room_types.c.room_type_name == df.loc[0, 'room_type'])
result = connection.execute(sel).fetchone()
print(result)

(2,)


Чтобы узнать, как выглядит запрос, отправляемый в базу данных, можно выполнить команду

In [33]:
print(str(sel))
print(sel.compile().params)

SELECT room_types.room_type_id 
FROM room_types 
WHERE room_types.room_type_name = :room_type_name_1
{'room_type_name_1': 'Hotel room'}


### Задание 4.1.1. 
Наполните данными таблицу *listings*

In [110]:
lst = []
for row_id in df.index[:]:
    
    
    # Определите значение идентификатора района. 
    # Присвойте ему имя current_neigh_id
    # Ваш код здесь
    sel = select([neighbourhoods.c.neigh_id])
    sel = sel.where(neighbourhoods.c.neigh_name==df.loc[row_id, 'neighbourhood'])
    current_neigh_id = connection.execute(sel).fetchone()[0]
    
    
    # Определите значение идентификатора типа комнаты. 
    # Присвойте ему имя current_room_type_id
    # Ваш код здесь
    sel = select([room_types.c.room_type_id])
    sel = sel.where(room_types.c.room_type_name==df.loc[row_id, 'room_type'])
    current_room_type_id = connection.execute(sel).fetchone()[0]
    
    
    # Значение идентификатора владельца можно взять напрямую из записи набора данных
    current_host_id = df.loc[row_id, 'host_id']
    
    
    tmp_dict={
        'listing_id': int(df.loc[row_id, 'id']),
        'listing_name': df.loc[row_id, 'name'],
        'host_id': int(current_host_id),
        'neighbourhood_id': int(current_neigh_id),
        'latitude': float(df.loc[row_id, 'latitude']),
        'longitude': float(df.loc[row_id, 'longitude']),
        'room_type_id': int(current_room_type_id),
        'price': int(df.loc[row_id, 'price']),
        'minimum_nights': int(df.loc[row_id, 'minimum_nights']),
        'number_of_reviews': int(df.loc[row_id, 'number_of_reviews']),
        'last_review': dt.strptime(df.loc[row_id, 'last_review'],'%Y-%m-%d'),
        'reviews_per_month': df.loc[row_id, 'reviews_per_month'],
        'availability_365': int(df.loc[row_id, 'availability_365'])
    }    
    lst.append(tmp_dict)

In [35]:
lst[0]

{'listing_id': 50904,
 'listing_name': 'aplace/antwerp: cosy suite - fashion district',
 'host_id': 234077,
 'neighbourhood_id': 19,
 'latitude': 51.21825,
 'longitude': 4.39901,
 'room_type_id': 2,
 'price': 145,
 'minimum_nights': 1,
 'number_of_reviews': 1,
 'last_review': datetime.datetime(2015, 5, 6, 0, 0),
 'reviews_per_month': 0.01,
 'availability_365': 226}

In [36]:
ins=listings.insert()
result=connection.execute(ins, lst)

Проверка с помощью запроса

In [37]:
sel=select([listings])
rp=connection.execute(sel).fetchone()
pprint(rp)

(50904, 'aplace/antwerp: cosy suite - fashion district', 234077, 19, Decimal('51.22'), Decimal('4.40'), 2, 145, 1, 1, datetime.date(2015, 5, 6), Decimal('0.01'), 226)


C:\Users\myfri\anaconda3\lib\site-packages\sqlalchemy\sql\sqltypes.py:666: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


In [38]:
l=df.loc[0,'latitude']
print(l, type(l), type(float(l)))

51.21825 <class 'numpy.float64'> <class 'float'>


## 4.2 Таблицы *users, orders, line_items*

### Задание 4.2.1.
Наполните данными таблицы *users, orders, line_items*

In [39]:
def insert_values(table, values_list):
    for value in values_list:
        ins = insert(table).values(value)
        connection.execute(ins)

In [40]:
user_list=[(1,'Nicolas','nicolas@rambler.ru','+7-929-616-88-77','@#$%890'),
           (2,'Lida','lidaok@gmail.com','+7-929-616-88-77','yyT$%333'),
           (3,'Vera','lveramuns@gmail.com','+7-353-214-12-90','yyT$%333'),
           (4,'Ivan','ivaturgenev@yandex.ru','+7-047-121-89-95','tT6^7&#20Oy'),
           (5,'Svetlana','svetaivanova@microsoft.com','+7-812-555-48-71','SD%@OUsdc7')
          ]
insert_values(users,user_list)

In [41]:
sel=select([users])
connection.execute(sel).fetchone()

(1, 'Nicolas', 'nicolas@rambler.ru', '+7-929-616-88-77', '@#$%890', datetime.date(2021, 5, 6), datetime.date(2021, 5, 6))

In [42]:
orders_list=[(1,1,True,500),
             (2,2,True,150),
             (3,3,True,180),
             (4,1,True,200),
             (5,2,True,220)]
insert_values(orders,orders_list)

In [43]:
sel=select([orders])
connection.execute(sel).fetchone()

(1, 1)

## 5. Выполните задания
<img src="./Img/Antwerpen_6b.png">

Выполните следующие задания с помощью запросов к базе данных *Antwerp.db*

1. Распечатать названия районов и количество объектов в каждом районе. Упорядочить результат по убыванию числа объектов.
2. Распечатать название района, имя владельца, название объекта для всех объектов размещения в районах *Dam* и *Donk*. Отсортировать результат по названию района и в каждом районе по имени владельца.
3. Распечатать имя владельца и число объектов, которым он владеет, для всех владельцев, которые владеют семью и более объектами. Результат отсортировать по числу объектов в порядке убывания.
4. Сколько комнат каждого типа в районе *Central Station?*
5. Распечатать имена вледельцев и число объектов каждого владельца для всех владельцев, владеющих двумя и более объектами.
6. Комнатами какого типа владеет *Mari?*. Распечатать число комнат каждого типа.
7. Распечатать все объекты, принадлежащие владельцам *Mari* и *Marie*. Выполнить этот запрос с помощью подзапроса и без использования JOIN.  
8. Найти объект размещения максимальной цены.
9. Найти среднюю цену объекта в районе *Hoogte*.
9. Распечатать названия и цены объектов в районе *Hoogte* с ценой ниже средней цены по району *Hoogte*.